In [ ]:
import requests
import json

# Set the Amadeus API endpoint URL
url = "https://test.api.amadeus.com/v2/shopping/flight-offers"

# Set the required parameters for the flight-offers API
params = {
    "originLocationCode": "BEG",
    "destinationLocationCode": "JFK",
    "departureDate": "2023-05-01",
    "returnDate": "2023-05-30",
    "adults": 1,
    "currencyCode": "USD",
    "max": 1
}

# Set the Amadeus API credentials
client_id = "client_id"
client_secret = "client_secret"

# Get the access token from Amadeus using client_id and client_secret
auth_url = "https://test.api.amadeus.com/v1/security/oauth2/token"
auth_data = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret
}
auth_response = requests.post(auth_url, data=auth_data)
auth_token = auth_response.json()["access_token"]

# Set the authorization header with the access token
headers = {
    "Authorization": f"Bearer {auth_token}"
}

# Make the API request to get the cheapest available flight offer
response = requests.get(url, params=params, headers=headers)
response_data = json.loads(response.text)

# Extract the cheapest flight offer from the response
if "data" in response_data:
    cheapest_price = response_data["data"][0]["price"]["total"]
    cheapest_route = response_data["data"][0]["itineraries"][0]["segments"]
    print(f"The cheapest available flight offer from UFA to BEG is {cheapest_price} USD on the following route:")
    for segment in cheapest_route:
        dep_iata_code = segment['departure']['iataCode']
        dep_datetime = segment['departure']['at'][0:10] + " " + segment['departure']['at'][11:16]
        arr_iata_code = segment['arrival']['iataCode']
        arr_datetime = segment['arrival']['at'][0:10] + " " + segment['arrival']['at'][11:16]
        flight_number = f"{segment['carrierCode']}-{segment['number']}"
        print(f"    {dep_iata_code} {dep_datetime} -> {arr_iata_code} {arr_datetime} | {flight_number}")
else:
    print("No flight offers found. Check your parameters and try again.")
